#SetUp

In [ ]:
%cd "/content/drive/MyDrive/Diplo2"
%env PYTHONPATH =
!pip install virtualenv
!virtualenv semgs
#!wget https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local python=3.6 ujson

In [ ]:
%cd "/content/drive/MyDrive/Diplo2"
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')
import os
os.environ['CONDA_PREFIX'] = '/content/drive/MyDrive/Diplo2/semgs'
!pip install tensorflow==1.13.1
!pip install keras==2.2.4
!pip install numpy==1.16.2
!pip install scipy==1.2.1
!pip install math
!pip install matplotlib
!pip install scikit-learn==0.20.3
!pip install virtualenv
!virtualenv semgs

In [ ]:
import glob
from scipy.io import loadmat
import os
import numpy as np
import time
import matplotlib.pyplot as plt


#separate semg

In [ ]:
import glob
from scipy.io import loadmat
import numpy as np
import time
import matplotlib.pyplot as plt


for s in range (1,28):
    DATA = []
    gesture_number = 0
    for file in sorted(glob.glob("/content/drive/MyDrive/Diplo2/Ninapro/s{}/*.mat".format(s))):
        data = loadmat(file)
        emg = data['emg']
        restimulus = data['restimulus']
        rerepetition = data['rerepetition']

        print("EMG size {}".format(emg.shape))

        plt.subplot(3,1,1)
        plt.plot(emg)
        plt.subplot(3,1,2)
        plt.plot(restimulus)
        plt.subplot(3,1,3)
        plt.plot(rerepetition)
        plt.show()

        L = len(restimulus)

        previous_gesture = 0
        current_signal = np.array([]).astype(np.float32)

        for i in range(L):
            current_gesture = restimulus[i]

            if current_gesture != 0:
                if previous_gesture == 0:
                    gesture_number += (rerepetition[i].item() == 1)
                    current_signal = np.append(current_signal, emg[i])
                else:
                    current_signal = np.vstack((current_signal, emg[i]))

            elif (current_gesture == 0 and previous_gesture != 0) or (i == L - 1 and current_gesture != 0):
                DATA.append({
                    "emg": current_signal,
                    "repetition": rerepetition[i-1].item(),
                    "stimulus": gesture_number
                })
                # name = getSubjectKey(sub, gesture_number, rerepetition[i - 1].item())
                # save_signal = {
                #     "emg": current_signal,
                # }

                # if rerepetition[i-1] != 2 and rerepetition[i-1] != 5 and rerepetition[i-1] != 7:
                #     savemat(folder_3 + "/S{}_G{}_R{}.mat".format(sub, gesture_number, rerepetition[i - 1].item()), save_signal)
                # else:
                #     savemat(folder_4 + "/S{}_G{}_R{}.mat".format(sub, gesture_number, rerepetition[i - 1].item()), save_signal)

                current_signal = np.array([]).astype(np.float32)
            previous_gesture = current_gesture
    DATA_TRAIN = [x for x in DATA if x["repetition"] not in [2,5,7]]
    DATA_TEST = [x for x in DATA if x["repetition"] in [2,5,7]]
    np.save("/content/drive/MyDrive/Diplo2/Data/DATA_TRAIN{}".format(s),DATA_TRAIN)
    np.save("/content/drive/MyDrive/Diplo2/Data/DATA_TEST{}".format(s),DATA_TEST)

#Select Subject

In [ ]:
s=1
DATA_TRAIN=np.load("/content/drive/MyDrive/Diplo2/Data/DATA_TRAIN{}.npy".format(s), allow_pickle=True)
DATA_TEST=np.load("/content/drive/MyDrive/Diplo2/Data/DATA_TEST{}.npy".format(s), allow_pickle=True)
print(len(DATA_TRAIN))
print(len(DATA_TEST))
print(DATA_TRAIN.shape)

#Preprocessing

In [ ]:
WINDOW_SIZE = 15
WINDOW_SLIDE = 5

In [ ]:
import scipy.signal
import math
def lpf(x):
    f = 1 / 50
    b, a = scipy.signal.butter(1, f, 'low')
    output = scipy.signal.filtfilt(b, a, x, axis=0)
    return output

def noise(x, snr_l=25):
    snr_u = 45
    db = np.random.randint(snr_l, snr_u, (1,))[0]
    snr = 10 ** (db / 10)
    Xp = np.sum(x ** 2, axis=0) / x.shape[0]
    Np = Xp / snr
    n = np.random.normal(size=x.shape, scale=np.sqrt(Np))
    xn = x + n
    return xn

def mlaw(x):
  x = (x - np.min(x)) / (np.max(x) - np.min(x))*2 - 1
  m = 32
  # Apply the mlaw logic element-wise
  mlaw_func = np.vectorize(lambda val: (math.log(1 + m * abs(val)) / math.log(1 + m)) * math.copysign(1, val))
  return mlaw_func(x)

l=180
for x in DATA_TRAIN:
    emg = x["emg"]
    if l > len(emg):
        l = len(emg)
for x in DATA_TEST:
    emg = x["emg"]
    if l > len(emg):
        l = len(emg)
print(l)

In [ ]:
emg = DATA_TRAIN[86]["emg"]
plt.plot(emg)
plt.show()
print(DATA_TRAIN[86]["stimulus"])

In [ ]:
plt.plot((emg - np.min(emg)) / (np.max(emg) - np.min(emg)))
emg= (emg - np.min(emg)) / (np.max(emg) - np.min(emg))

In [ ]:

emg = np.array(emg)
print(emg.shape)
cut = int((len(emg)-l)/2)
for i in range(cut):
    emg=np.delete(emg, 0, 0)
for i in range (len(emg)-l):
    emg=np.delete(emg, l, 0)
plt.plot(emg)


In [ ]:
emg=lpf(emg)
plt.plot(emg)

In [ ]:
emg=mlaw(emg)
plt.plot(emg)

In [ ]:
emg = noise(emg)
print (emg.shape)
plt.plot(emg)

In [ ]:
import scipy.signal
X_train, y_train = [], []
def lpf(x):
    f = 1 / 50
    b, a = scipy.signal.butter(1, f, 'low')
    output = scipy.signal.filtfilt(b, a, x, axis=0)
    return output

def noise(x, snr_l=25):
    snr_u = 45
    db = np.random.randint(snr_l, snr_u, (1,))[0]
    snr = 10 ** (db / 10)
    Xp = np.sum(x ** 2, axis=0) / x.shape[0]
    Np = Xp / snr
    n = np.random.normal(size=x.shape, scale=np.sqrt(Np))
    xn = x + n
    return xn

def mlaw(x):
  m = 32
  if m==0: return x


  #x = (x - np.min(x)) / (np.max(x) - np.min(x))
  # Apply the mlaw logic element-wise
  mlaw_func = np.vectorize(lambda val: (math.log(1 + m * abs(val)) / math.log(1 + m)) * math.copysign(1, val))
  return mlaw_func(x)


aug = 4

z=0
for j in range (aug):
    for x in DATA_TRAIN:

        emg = x["emg"]
        if(len(emg)>l):
          z=z+1
          cut = int((len(emg)-l)/2)
          for i in range(cut):
            emg=np.delete(emg, 0, 0)
          for i in range (len(emg)-l):
            emg=np.delete(emg, l, 0)
        emg = lpf(emg)
        emg = mlaw(emg)
        emg = (emg - np.min(emg)) / (np.max(emg) - np.min(emg))
        emg = noise(emg)
        for i in range(0, len(emg)-WINDOW_SIZE, WINDOW_SLIDE):
            X_train.append(emg[i:i+WINDOW_SIZE, :])
            y_train.append(x["stimulus"])
print(z)
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train.shape, y_train.shape

In [ ]:
X_test, y_test = [], []

for x in DATA_TEST:
    emg = x["emg"]
    if(len(emg)>l):
          z=z+1
          cut = int((len(emg)-l)/2)
          for i in range(cut):
            emg=np.delete(emg, 0, 0)
          for i in range (len(emg)-l):
            emg=np.delete(emg, l, 0)
    emg = lpf(emg)
    emg = mlaw(emg)
    emg = (emg - np.min(emg)) / (np.max(emg) - np.min(emg))
    emg = noise(emg)
    for i in range(0, len(emg)-WINDOW_SIZE, WINDOW_SLIDE):
        X_test.append(emg[i:i+WINDOW_SIZE, :])
        y_test.append(x["stimulus"])
X_test = np.array(X_test)
y_test = np.array(y_test)

X_test.shape, y_test.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(np.unique(y_train))
le.classes_

In [ ]:
!source /content/drive/MyDrive/Diplo2/semgs/bin/activate
from keras.utils import to_categorical

# Labels 1-52 should be mapped to 0-51 and then encoded
y_train = to_categorical(le.transform(y_train), 52)
y_test = to_categorical(le.transform(y_test), 52)


In [ ]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
X_train.shape, y_train.shape


In [ ]:
X_test.shape, y_test.shape

In [ ]:
import random


indices = np.random.choice(len(X_train), len(X_train), False)
X_train = X_train[indices]
y_train = y_train[indices]

indices = np.random.choice(len(X_test), len(X_test), False)
X_test = X_test[indices]
y_test = y_test[indices]



In [ ]:
from keras.metrics import *

def top3acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

def top5acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

#Majority Vote

In [ ]:
X_maj, y_maj = [], []

z=0
for x in DATA_TEST:
        emg = x["emg"]
        emg = lpf(emg)
        emg = mlaw(emg)
        emg = (emg - np.min(emg)) / (np.max(emg) - np.min(emg))
        emg = noise(emg)
        if(len(emg)>l):
          cut = int((len(emg)-l)/2)
          for i in range(cut):
            emg=np.delete(emg, 0, 0)
          for i in range (len(emg)-l):
            emg=np.delete(emg, l, 0)

        X_maj.append(emg)
        y_maj.append(x)

X_maj = np.array(X_maj)
y_maj = np.array(y_maj)
X_maj.shape, y_maj.shape

In [ ]:
def windowed_array(arr, size=WINDOW_SIZE, slide=WINDOW_SLIDE):
    W= len(arr)
    windows = []
    for i in range(0, W - size, slide):
        window = arr[i:i + size]
        windows.append(window[:, :, np.newaxis])
    return np.stack(windows)

In [ ]:
from keras.utils import Sequence
from scipy.stats import mode
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from keras.metrics import *
from keras.utils import Sequence
import keras
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle

class CustomSequenceGenerator(Sequence):
    def __init__(self, data_list, labels, batch_size=4, num_classes=52, shuffle_data=True):
        self.data_list = data_list
        self.labels = labels
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle_data = shuffle_data
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.data_list) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x = [windowed_array(self.data_list[i]) for i in batch_indices]
        batch_y_dicts = [self.labels[i] for i in batch_indices] # Get the list of dictionaries

        # Stack all windows together for training
        batch_x = np.concatenate(batch_x, axis=0)

        # Extract the 'stimulus' value from each dictionary and repeat for each window
        batch_y_labels = [y_dict['stimulus'] for y_dict in batch_y_dicts]
        # Adjust labels to be zero-indexed
        batch_y_repeated = np.repeat([label - 1 for label in batch_y_labels], batch_x.shape[0] // len(batch_y_labels)) # Repeat label for each window


        return batch_x, tf.keras.utils.to_categorical(batch_y_repeated, num_classes=self.num_classes) # Use the extracted labels

    def on_epoch_end(self):
        self.indices = np.arange(len(self.data_list))
        if self.shuffle_data:
            self.indices = shuffle(self.indices)

In [ ]:
def evaluate_with_majority_voting(model, X_test, y_test, loss_fn=keras.losses.CategoricalCrossentropy()):
    """
    Evaluate model performance using majority voting on a test set, and also compute loss."""

    y_pred = []
    true_labels_list = []
    total_loss = 0
    y_prob_avv = []
    num_samples = len(X_test)
    num_classes = model.output_shape[-1]

    # Loop through each test sample
    for array, y_dict in zip(X_test, y_test):
        # Window the array
        windows = windowed_array(array)  # Shape: [num_windows, 15, 10, 1]

        # Get model predictions
        probs = model.predict(windows, verbose=0)  # Shape: [num_windows, 52]

        # Compute per-window predictions (argmax over classes)
        preds = np.argmax(probs, axis=1)

        # Majority voting
        voted = mode(preds, keepdims=False).mode
        y_pred.append(voted)

        # Extract and adjust the true label
        true_label = y_dict['stimulus'] - 1 # Adjust label to be zero-indexed
        true_labels_list.append(true_label)
        avg_prob = np.mean(probs, axis=0)
        y_prob_avv.append(avg_prob)
        # Calculate loss for the current sample using the true label for its window
        true_label_repeated = np.repeat(true_label, windows.shape[0])
        # Adjust labels to be zero-indexed before to_categorical
        true_label_one_hot = tf.keras.utils.to_categorical(true_label_repeated, num_classes=52)
        batch_loss = loss_fn(true_label_one_hot, probs).numpy()  # loss per window
        total_loss += np.mean(batch_loss) # Take the mean loss across windows for the sample

    # Compute average loss for the entire test set
    avg_loss = total_loss / num_samples
    true_labels_one_hot = tf.keras.utils.to_categorical(true_labels_list, num_classes=num_classes)
    # Compute majority voting accuracy using the adjusted true labels
    acc = accuracy_score(true_labels_list, y_pred)
    top3acc_tensor = tf.keras.metrics.top_k_categorical_accuracy(true_labels_one_hot, np.array(y_prob_avv), k=3)
    top3acc = np.mean(top3acc_tensor.numpy())
    top5acc_tensor = tf.keras.metrics.top_k_categorical_accuracy(true_labels_one_hot, np.array(y_prob_avv), k=5)
    top5acc = np.mean(top5acc_tensor.numpy())
    print(f"Majority Voting Test Accuracy: {acc:.4f}")
    print(f"Majority Voting Test Top-3 Accuracy: {top3acc:.4f}")
    print(f"Majority Voting Test Top-5 Accuracy: {top5acc:.4f}")
    print(f"Average Test Loss: {avg_loss:.4f}")
    return acc, avg_loss, top3acc, top5acc

In [ ]:
def predict_with_majority_voting(model, X_data,y_true):
    """
    Predict class for each full array using majority vote over its windows.

    """
    y_pred = []

    for array in X_data:
        windows = windowed_array(array)  # shape: (num_windows, 15, 10, 1)
        probs = model.predict(windows, verbose=0)
        preds = np.argmax(probs, axis=1)
        voted = mode(preds, keepdims=False).mode
        y_pred.append(voted)

    acc = accuracy_score(y_true, y_pred)
    print(f"Predict Majority Voting Accuracy: {acc:.4f}")

    return np.array(y_pred)

#ATZORINET WITH RNN

#ATZORINET

In [ ]:
import keras
from keras.layers import *
from keras import initializers, regularizers
from keras.models import Model


def Atzorinet(input_shape, classes = 52, l2 = 0.0005, dropout = 0.2, batch_norm = True):
    #creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)


    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    X = ZeroPadding2D((0, 4))(X)
    X = Conv2D(32, (1, 10), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block1_conv2d_32_1_10')(X)
    X = Activation('relu', name='block1_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block1_dropout')(X)

    # Block 2
    X = ZeroPadding2D((1, 1))(X)
    X = Conv2D(32, (3, 3), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block2_conv2d_32_3_3')(X)
    X = Activation('relu', name='block2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block2_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block2_average_pool')(X)

    # Block 3
    X = ZeroPadding2D((2, 2))(X)
    X = Conv2D(64, (5, 5), padding='valid',#kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block3_conv2d_64_5_5')(X)
    X = Activation('relu', name='block3_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block3_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block3_pool')(X)

    # Block 4
    X = ZeroPadding2D((2, 0))(X)
    X = Conv2D(64, (5, 1), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block4_conv2d_64_5x1')(X)
    X = Activation('relu', name='block4_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block4_dropout')(X)

    # Block 5
    X = Conv2D(classes, (1, 1), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block5_conv2d_{}_1x1'.format(classes))(X)
    X = Activation('softmax', name='block5_soft')(X)
    X = Flatten()(X)

    model = Model(inputs=input, outputs=X, name='Atzori')

    return model

model = Atzorinet((15,10,1))
model.summary()

In [ ]:
from keras.layers import *
from keras import initializers, regularizers
from keras.models import Model


def AtzorinetRNN_Bid(input_shape, classes = 52, l2 = 0.0005, dropout = 0.2, batch_norm = True):
    #creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)


    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    X = ZeroPadding2D((0, 4))(X)
    X = Conv2D(32, (1, 10), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block1_conv2d_32_1_10')(X)
    X = Activation('relu', name='block1_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block1_dropout')(X)

    # Block 2
    X = ZeroPadding2D((1, 1))(X)
    X = Conv2D(32, (3, 3), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block2_conv2d_32_3_3')(X)
    X = Activation('relu', name='block2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block2_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block2_average_pool')(X)

    # Block 3
    X = ZeroPadding2D((2, 2))(X)
    X = Conv2D(64, (5, 5), padding='valid',#kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block3_conv2d_64_5_5')(X)
    X = Activation('relu', name='block3_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block3_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block3_pool')(X)

    # Block 4
    X = ZeroPadding2D((2, 0))(X)
    X = Conv2D(64, (5, 1), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block4_conv2d_64_5x1')(X)
    X = Activation('relu', name='block4_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block4_dropout')(X)

    # Block 5
    time_steps = X.shape[1] * X.shape[2]  # Combine spatial dimensions
    features = X.shape[3]  # Channels
    X = Reshape((time_steps, features))(X)
    X = Reshape((2,32))(X)
    lstm_layer = LSTM(128, return_sequences=True, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul, name='rnn_lstm')
    X = Bidirectional(lstm_layer)(X)
    X = Activation('relu', name='block5_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block5_dropout')(X)
    lstm_layer_2 = LSTM(64, return_sequences=False, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul, name='rnn_lstm_2')
    X = Bidirectional(lstm_layer_2)(X)
    X = Activation('relu', name='block5_2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block5_2_dropout')(X)

    # Block 6
    X = Dense(classes, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block6_Dense'.format(classes))(X)
    X = Activation('softmax', name='block6_soft')(X)
    # Create Model
    model = Model(inputs=input, outputs=X, name='Atzori_with_RNN')

    return model
model1 = AtzorinetRNN_Bid((15,10,1))
model1.summary()

In [ ]:
from keras.layers import *
from keras import initializers, regularizers
from keras.models import Model


def AtzorinetRNN(input_shape, classes = 52, l2 = 0.0005, dropout = 0.2, batch_norm = True):
    #creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)


    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    X = ZeroPadding2D((0, 4))(X)
    X = Conv2D(32, (1, 10), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block1_conv2d_32_1_10')(X)
    X = Activation('relu', name='block1_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block1_dropout')(X)

    # Block 2
    X = ZeroPadding2D((1, 1))(X)
    X = Conv2D(32, (3, 3), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block2_conv2d_32_3_3')(X)
    X = Activation('relu', name='block2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block2_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block2_average_pool')(X)

    # Block 3
    X = ZeroPadding2D((2, 2))(X)
    X = Conv2D(64, (5, 5), padding='valid',#kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block3_conv2d_64_5_5')(X)
    X = Activation('relu', name='block3_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block3_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block3_pool')(X)

    # Block 4
    X = ZeroPadding2D((2, 0))(X)
    X = Conv2D(64, (5, 1), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block4_conv2d_64_5x1')(X)
    X = Activation('relu', name='block4_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block4_dropout')(X)

    # Block 5
    time_steps = X.shape[1] * X.shape[2]  # Combine spatial dimensions
    features = X.shape[3]  # Channels
    X = Reshape((time_steps, features))(X)
    X = Reshape((2,32))(X)
    lstm_layer = LSTM(128, return_sequences=True, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul, name='rnn_lstm')
    X = lstm_layer(X)
    X = Activation('relu', name='block5_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block5_dropout')(X)
    lstm_layer_2 = LSTM(64, return_sequences=False, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul, name='rnn_lstm_2')
    X = lstm_layer_2(X)
    X = Activation('relu', name='block5_2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block5_2_dropout')(X)

    # Block 6
    X = Dense(classes, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block6_Dense'.format(classes))(X)
    X = Activation('softmax', name='block6_soft')(X)
    # Create Model
    model = Model(inputs=input, outputs=X, name='Atzori_with_RNN')

    return model
model1 = AtzorinetRNN((15,10,1))
model1.summary()

#Time Distributed with RNN

In [ ]:
from keras.layers import *
from keras import initializers, regularizers
from keras.models import Model

def Atzorinet2(input_shape=(5, 10, 1), classes=52, l2=0.0005, dropout=0.2, batch_norm=True):
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)

    X_input = Input(input_shape, name='block0_input')
    X = X_input

    # Block 1: Conv on full width
    X = Conv2D(32, (1, 10), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul)(X)
    X = Activation('relu')(X)
    if batch_norm: X = BatchNormalization()(X)
    if dropout > 0: X = Dropout(dropout)(X)

    # Block 2: 3x3 conv, with SAME padding
    X = Conv2D(32, (3, 3), padding='same', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul)(X)
    X = Activation('relu')(X)
    if batch_norm: X = BatchNormalization()(X)
    if dropout > 0: X = Dropout(dropout)(X)
    # Removed pooling to preserve size

    # Block 3: 3x3 conv, same padding
    X = Conv2D(64, (3, 3), padding='same', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul)(X)
    X = Activation('relu')(X)
    if batch_norm: X = BatchNormalization()(X)
    if dropout > 0: X = Dropout(dropout)(X)
    # Removed pooling again to preserve (5,1) height

    # Block 4: Final Conv (5x1) — keep this
    X = Conv2D(64, (5, 1), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul)(X)
    X = Activation('relu')(X)


    # Output: flatten
    X = Flatten()(X)

    model = Model(inputs=X_input, outputs=X, name='Atzori2')
    return model
model = Atzorinet2((5,10,1))
model.summary()

In [ ]:
def Atzorinet_Time_Bidir(input_shape, classes=52, l2=0.0005, dropout=0.2, batch_norm=True):
    # creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)

    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    atzorinet_layer = Atzorinet2((5, 10, 1))
    X = Reshape((3, 5, 10, 1))(X)
    X = TimeDistributed(atzorinet_layer)(X)

    # Block LSTM
    # Reshape to combine spatial dimensions and maintain 3D input for LSTM
    shape_x = X.shape
    # The output of TimeDistributed is (None, 3, 1, 1, 64).
    # We want to reshape to (None, 3, 64) for the LSTM.
    #X = Reshape((shape_x[1], shape_x[4]))(X)  # Reshape to (None, 3, 64)
    X = Reshape ((6,32))(X)
    lstm_layer = LSTM(128, return_sequences=True, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                      name='rnn_lstm')
    X = Bidirectional(lstm_layer)(X)
    X = Activation('relu', name='LSTM_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_dropout')(X)
    lstm_layer_2 = LSTM(64, return_sequences=False, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                        name='rnn_lstm_2')
    X = Bidirectional(lstm_layer_2)(X)
    X = Activation('relu', name='LSTM_2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_2_dropout')(X)

    # Block 2
    X = Dense(classes, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
              name='block2_Dense'.format(classes))(X)
    X = Activation('softmax', name='block2_soft')(X)

    model = Model(inputs=input, outputs=X, name='Atzori_Time_Bidir')
    return model
model = Atzorinet_Time_Bidir((15,10,1))
model.summary()

In [ ]:
def Atzorinet_Time(input_shape, classes=52, l2=0.0005, dropout=0.2, batch_norm=True):
    # creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)

    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    atzorinet_layer = Atzorinet2((5, 10, 1))
    X = Reshape((3, 5, 10, 1))(X)
    X = TimeDistributed(atzorinet_layer)(X)

    # Block LSTM
    # Reshape to combine spatial dimensions and maintain 3D input for LSTM
    shape_x = X.shape
    X = Reshape ((6,32))(X)
    # The output of TimeDistributed is (None, 3, 1, 1, 64).
    # We want to reshape to (None, 3, 64) for the LSTM.
    #X = Reshape((shape_x[1], shape_x[4]))(X)  # Reshape to (None, 3, 64)

    lstm_layer = LSTM(128, return_sequences=True, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                      name='rnn_lstm')
    X = lstm_layer(X)
    X = Activation('relu', name='LSTM_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_dropout')(X)
    lstm_layer_2 = LSTM(64, return_sequences=False, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                        name='rnn_lstm_2')
    X = lstm_layer_2(X)
    X = Activation('relu', name='LSTM_2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_2_dropout')(X)

    # Block 2
    X = Dense(classes, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
              name='block2_Dense'.format(classes))(X)
    X = Activation('softmax', name='block2_soft')(X)

    model = Model(inputs=input, outputs=X, name='Atzori_Time')
    return model
model = Atzorinet_Time((15,10,1))
model.summary()

#Time Distributed RNN with 45 window

In [ ]:

from keras.layers import *
from keras import initializers, regularizers
from keras.models import Model


def Atzorinet3(input_shape, classes = 52, l2 = 0.0005, dropout = 0.2, batch_norm = True):
    #creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)


    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    X = ZeroPadding2D((0, 4))(X)
    X = Conv2D(32, (1, 10), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block1_conv2d_32_1_10')(X)
    X = Activation('relu', name='block1_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block1_dropout')(X)

    # Block 2
    X = ZeroPadding2D((1, 1))(X)
    X = Conv2D(32, (3, 3), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block2_conv2d_32_3_3')(X)
    X = Activation('relu', name='block2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block2_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block2_average_pool')(X)

    # Block 3
    X = ZeroPadding2D((2, 2))(X)
    X = Conv2D(64, (5, 5), padding='valid',#kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block3_conv2d_64_5_5')(X)
    X = Activation('relu', name='block3_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='block3_dropout')(X)
    X = MaxPooling2D((3, 3), strides=(3, 3), name='block3_pool')(X)

    # Block 4
    X = ZeroPadding2D((2, 0))(X)
    X = Conv2D(64, (5, 1), padding='valid', kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
               name='block4_conv2d_64_5x1')(X)
    X = Activation('relu', name='block4_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)

    X = Flatten()(X)

    model = Model(inputs=input, outputs=X, name='Atzori')

    return model

model = Atzorinet3((15,10,1))
model.summary()

In [ ]:
def Atzorinet_Time_Bidir2(input_shape, classes=52, l2=0.0005, dropout=0.2, batch_norm=True):
    # creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)

    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    atzorinet_layer = Atzorinet3((15, 10, 1))
    X = Reshape((3, 15, 10, 1))(X)
    X = TimeDistributed(atzorinet_layer)(X)

    # Block LSTM
    # Reshape to combine spatial dimensions and maintain 3D input for LSTM
    shape_x = X.shape
    # The output of TimeDistributed is (None, 3, 1, 1, 64).
    # We want to reshape to (None, 3, 64) for the LSTM.
    #X = Reshape((shape_x[1], shape_x[4]))(X)  # Reshape to (None, 3, 64)
    X = Reshape ((6,32))(X)
    lstm_layer = LSTM(128, return_sequences=True, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                      name='rnn_lstm')
    X = Bidirectional(lstm_layer)(X)
    X = Activation('relu', name='LSTM_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_dropout')(X)
    lstm_layer_2 = LSTM(64, return_sequences=False, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                        name='rnn_lstm_2')
    X = Bidirectional(lstm_layer_2)(X)
    X = Activation('relu', name='LSTM_2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_2_dropout')(X)

    # Block 2
    X = Dense(classes, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
              name='block2_Dense'.format(classes))(X)
    X = Activation('softmax', name='block2_soft')(X)

    model = Model(inputs=input, outputs=X, name='Atzori_Time_Bidir')
    return model
model = Atzorinet_Time_Bidir2((45,10,1))
model.summary()

In [ ]:
def Atzorinet_Time2(input_shape, classes=52, l2=0.0005, dropout=0.2, batch_norm=True):
    # creating the CNN
    kernel_regul = regularizers.l2(l2)
    kernel_init = initializers.glorot_normal(seed=0)

    # Block 0
    input = Input(input_shape, name='block0_input')
    X = input

    # Block 1
    atzorinet_layer = Atzorinet3((15, 10, 1))
    X = Reshape((3, 15, 10, 1))(X)
    X = TimeDistributed(atzorinet_layer)(X)

    # Block LSTM
    # Reshape to combine spatial dimensions and maintain 3D input for LSTM
    shape_x = X.shape
    # The output of TimeDistributed is (None, 3, 1, 1, 64).
    # We want to reshape to (None, 3, 64) for the LSTM.
    #X = Reshape((shape_x[1], shape_x[4]))(X)  # Reshape to (None, 3, 64)
    X = Reshape ((6,32))(X)
    lstm_layer = LSTM(128, return_sequences=True, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                      name='rnn_lstm')
    X = lstm_layer(X)
    X = Activation('relu', name='LSTM_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_dropout')(X)
    lstm_layer_2 = LSTM(64, return_sequences=False, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
                        name='rnn_lstm_2')
    X = lstm_layer_2(X)
    X = Activation('relu', name='LSTM_2_relu')(X)
    if batch_norm:
        X = BatchNormalization()(X)
    if dropout > 0:
        X = Dropout(dropout, name='LSTM_2_dropout')(X)

    # Block 2
    X = Dense(classes, kernel_initializer=kernel_init, kernel_regularizer=kernel_regul,
              name='block2_Dense'.format(classes))(X)
    X = Activation('softmax', name='block2_soft')(X)

    model = Model(inputs=input, outputs=X, name='Atzori_Time')
    return model
model = Atzorinet_Time2((45,10,1))
model.summary()

#**Test**


#AtzoriNet

In [ ]:
import gc
import keras
import os
from keras import optimizers
from keras import callbacks
from keras.backend import clear_session
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import confusion_matrix

file = "/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet/"
if not os.path.exists(file):
    os.makedirs(file)
file = "/content/drive/MyDrive/Diplo2/Confusion/Atzorinet/"
if not os.path.exists(file):
    os.makedirs(file)




keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="auto", min_delta=0.001)
checkpoint_filepath="/content/drive/MyDrive/Diplo2/first_model.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', mode='min',
                                            save_best_only=True)

optimizer = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)



batch_size = [64,128,256,512]
epochs = [10,20,30,50]
batch_size = batch_size[2]
epochs = epochs[3]
loss = 'categorical_crossentropy'


model = Atzorinet((15,10,1))


model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', top3acc, top5acc])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback,model_checkpoint_callback], shuffle = True)
print("Evaluate on test data for Atzorinet with batch={} and epochs={}".format(batch_size, epochs))
model = keras.models.load_model("/content/drive/MyDrive/Diplo2/first_model.keras",custom_objects={'top3acc': top3acc, 'top5acc': top5acc})
results = model.evaluate(X_test, y_test, batch_size=1)
print("Atzorinet with batch={} and epochs={}".format(batch_size, epochs))
print("test loss, test ac, top3, top5:", results)
y_prediction = model.predict(X_test)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test_original = y_test
y_test=np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
cm = confusion_matrix(y_test, y_prediction)
cm=np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion/Atzorinet/{}".format(s),cm)
y_test = y_test_original
evaluate_with_majority_voting(model, X_maj, y_maj)
y_true = np.array([y_dict['stimulus'] - 1 for y_dict in y_maj])
y_val_pred = predict_with_majority_voting(model, X_maj, y_true)
cm = confusion_matrix(y_true, y_val_pred)
cm = np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet/{}".format(s),cm)

del model

#clear_session()
gc.collect()

#RNN

In [ ]:
import gc
import keras
from keras import optimizers
from keras import callbacks
from keras.backend import clear_session
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import confusion_matrix





keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="auto", min_delta=0.001)
checkpoint_filepath="/content/drive/MyDrive/Diplo2/first_model.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', mode='min',
                                            save_best_only=True)
optimizer = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)



batch_size = [64,128,256,512]
epochs = [10,20,30,50]
batch_size = batch_size[2]
epochs = epochs[3]
loss = 'categorical_crossentropy'


model = AtzorinetRNN((15,10,1))


model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', top3acc, top5acc])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback,model_checkpoint_callback], shuffle = True)
print("Evaluate on test data for AtzorinetRNN with batch={} and epochs={}".format(batch_size, epochs))
model = keras.models.load_model("/content/drive/MyDrive/Diplo2/first_model.keras",custom_objects={'top3acc': top3acc, 'top5acc': top5acc})
results = model.evaluate(X_test, y_test, batch_size=1)
print("AtzorinetRNN with batch={} and epochs={}".format(batch_size, epochs))
print("test loss, test ac, top3, top5:", results)
y_prediction = model.predict(X_test)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test_original = y_test
y_test=np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
cm = confusion_matrix(y_test, y_prediction)
cm=np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion/AtzorinetRNN/{}".format(s),cm)
y_test = y_test_original
evaluate_with_majority_voting(model, X_maj, y_maj)
y_true = np.array([y_dict['stimulus'] - 1 for y_dict in y_maj])
y_val_pred = predict_with_majority_voting(model, X_maj, y_true)
cm = confusion_matrix(y_true, y_val_pred)
cm = np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion_Majority/AtzorinetRNN/{}".format(s),cm)

del model

clear_session()
gc.collect()

#RNN BIDIR

In [ ]:
import gc
import keras
from keras import optimizers
from keras import callbacks
from keras.backend import clear_session
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import confusion_matrix




keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="auto", min_delta=0.001)
checkpoint_filepath="/content/drive/MyDrive/Diplo2/first_model.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', mode='min',
                                            save_best_only=True)
optimizer = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)



batch_size = [64,128,256,512]
epochs = [10,20,30,50]
batch_size = batch_size[2]
epochs = epochs[3]
loss = 'categorical_crossentropy'


model = AtzorinetRNN_Bid((15,10,1))


model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', top3acc, top5acc])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback,model_checkpoint_callback], shuffle = True)
print("Evaluate on test data for AtzorinetRNN with batch={} and epochs={}".format(batch_size, epochs))
model = keras.models.load_model("/content/drive/MyDrive/Diplo2/first_model.keras",custom_objects={'top3acc': top3acc, 'top5acc': top5acc})
results = model.evaluate(X_test, y_test, batch_size=1)
print("AtzorinetRNN with batch={} and epochs={}".format(batch_size, epochs))
print("test loss, test ac, top3, top5:", results)
y_prediction = model.predict(X_test)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test_original = y_test
y_test=np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
cm = confusion_matrix(y_test, y_prediction)
cm=np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion/AtzorinetRNN_Bid/{}".format(s),cm)
y_test = y_test_original
evaluate_with_majority_voting(model, X_maj, y_maj)
y_true = np.array([y_dict['stimulus'] - 1 for y_dict in y_maj])
y_val_pred = predict_with_majority_voting(model, X_maj, y_true)
cm = confusion_matrix(y_true, y_val_pred)
cm = np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion_Majority/AtzorinetRNN_Bid/{}".format(s),cm)

del model

#clear_session()
gc.collect()

#TIME RNN

In [ ]:
import gc
import keras
from keras import optimizers
from keras import callbacks
from keras.backend import clear_session
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import confusion_matrix

file = "/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time/"
if not os.path.exists(file):
    os.makedirs(file)
file = "/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time/"
if not os.path.exists(file):
    os.makedirs(file)



keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="auto", min_delta=0.001)
checkpoint_filepath="/content/drive/MyDrive/Diplo2/first_model.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', mode='min',
                                            save_best_only=True)
optimizer = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)



batch_size = [64,128,256,512]
epochs = [10,20,30,50]
batch_size = batch_size[2]
epochs = epochs[3]
loss = 'categorical_crossentropy'


model = Atzorinet_Time((15,10,1))


model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', top3acc, top5acc])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback,model_checkpoint_callback], shuffle = True)
print("Evaluate on test data for Atzorinet_Time with batch={} and epochs={}".format(batch_size, epochs))
model = keras.models.load_model("/content/drive/MyDrive/Diplo2/first_model.keras",custom_objects={'top3acc': top3acc, 'top5acc': top5acc})
results = model.evaluate(X_test, y_test, batch_size=1)
print("Atzorinet_Time with batch={} and epochs={}".format(batch_size, epochs))
print("test loss, test ac, top3, top5:", results)
y_prediction = model.predict(X_test)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test_original = y_test
y_test=np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
cm = confusion_matrix(y_test, y_prediction)
cm=np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time/{}".format(s),cm)
y_test = y_test_original
evaluate_with_majority_voting(model, X_maj, y_maj)
y_true = np.array([y_dict['stimulus'] - 1 for y_dict in y_maj])
y_val_pred = predict_with_majority_voting(model, X_maj, y_true)
cm = confusion_matrix(y_true, y_val_pred)
cm = np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time/{}".format(s),cm)

del model

clear_session()
gc.collect()

#Time Bid

In [ ]:
import gc
import keras
from keras import optimizers
from keras import callbacks
from keras.backend import clear_session
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import confusion_matrix
import os
file = "/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time/"
if not os.path.exists(file):
    os.makedirs(file)
file = "/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time/"
if not os.path.exists(file):
    os.makedirs(file)


keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="auto", min_delta=0.001)
checkpoint_filepath="/content/drive/MyDrive/Diplo2/first_model.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_loss', mode='min',
                                            save_best_only=True)
optimizer = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)



batch_size = [64,128,256,512]
epochs = [10,20,30,50]
batch_size = batch_size[2]
epochs = epochs[3]
loss = 'categorical_crossentropy'


model = Atzorinet_Time_Bidir((15,10,1))


model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', top3acc, top5acc])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback,model_checkpoint_callback], shuffle = True)
print("Evaluate on test data for Atzorinet_Time with batch={} and epochs={}".format(batch_size, epochs))
model = keras.models.load_model("/content/drive/MyDrive/Diplo2/first_model.keras",custom_objects={'top3acc': top3acc, 'top5acc': top5acc})
results = model.evaluate(X_test, y_test, batch_size=1)
print("Atzorinet_Time with batch={} and epochs={}".format(batch_size, epochs))
print("test loss, test ac, top3, top5:", results)
y_prediction = model.predict(X_test)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test_original = y_test
y_test=np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
cm = confusion_matrix(y_test, y_prediction)
cm=np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time_Bid/{}".format(s),cm)
y_test = y_test_original
evaluate_with_majority_voting(model, X_maj, y_maj)
y_true = np.array([y_dict['stimulus'] - 1 for y_dict in y_maj])
y_val_pred = predict_with_majority_voting(model, X_maj, y_true)
cm = confusion_matrix(y_true, y_val_pred)
cm = np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time_Bid/{}".format(s),cm)

del model

clear_session()
gc.collect()

#Preprocessing for 45

In [ ]:
WINDOW_SIZE = 45
WINDOW_SLIDE = 15

In [ ]:
import scipy.signal
import math
def lpf(x):
    f = 1 / 50
    b, a = scipy.signal.butter(1, f, 'low')
    output = scipy.signal.filtfilt(b, a, x, axis=0)
    return output

def noise(x, snr_l=25):
    snr_u = 45
    db = np.random.randint(snr_l, snr_u, (1,))[0]
    snr = 10 ** (db / 10)
    Xp = np.sum(x ** 2, axis=0) / x.shape[0]
    Np = Xp / snr
    n = np.random.normal(size=x.shape, scale=np.sqrt(Np))
    xn = x + n
    return xn

def mlaw(x):
  x = (x - np.min(x)) / (np.max(x) - np.min(x))*2 - 1
  m = 32
  # Apply the mlaw logic element-wise
  mlaw_func = np.vectorize(lambda val: (math.log(1 + m * abs(val)) / math.log(1 + m)) * math.copysign(1, val))
  return mlaw_func(x)

l=180
for x in DATA_TRAIN:
    emg = x["emg"]
    if l > len(emg):
        l = len(emg)
for x in DATA_TEST:
    emg = x["emg"]
    if l > len(emg):
        l = len(emg)
print(l)

In [ ]:
emg = DATA_TRAIN[0]["emg"]
plt.plot(emg)
plt.show()


In [ ]:

emg = np.array(emg)
print(emg.shape)
cut = int((len(emg)-l)/2)
for i in range(cut):
    emg=np.delete(emg, 0, 0)
for i in range (len(emg)-l):
    emg=np.delete(emg, l, 0)
plt.plot(emg)


In [ ]:
emg=lpf(emg)
plt.plot(emg)

In [ ]:
emg=mlaw(emg)
plt.plot(emg)

In [ ]:
plt.plot((emg - np.min(emg)) / (np.max(emg) - np.min(emg)))
emg= (emg - np.min(emg)) / (np.max(emg) - np.min(emg))

In [ ]:
emg = noise(emg)
print (emg.shape)
plt.plot(emg)

In [ ]:
import scipy.signal
X_train, y_train = [], []
def lpf(x):
    f = 1 / 50
    b, a = scipy.signal.butter(1, f, 'low')
    output = scipy.signal.filtfilt(b, a, x, axis=0)
    return output

def noise(x, snr_l=25):
    snr_u = 45
    db = np.random.randint(snr_l, snr_u, (1,))[0]
    snr = 10 ** (db / 10)
    Xp = np.sum(x ** 2, axis=0) / x.shape[0]
    Np = Xp / snr
    n = np.random.normal(size=x.shape, scale=np.sqrt(Np))
    xn = x + n
    return xn

def mlaw(x):
  m = 32
  if m==0: return x


  #x = (x - np.min(x)) / (np.max(x) - np.min(x))
  # Apply the mlaw logic element-wise
  mlaw_func = np.vectorize(lambda val: (math.log(1 + m * abs(val)) / math.log(1 + m)) * math.copysign(1, val))
  return mlaw_func(x)


aug = 4

z=0
for j in range (aug):
    for x in DATA_TRAIN:

        emg = x["emg"]
        emg = lpf(emg)
        emg = mlaw(emg)
        emg = (emg - np.min(emg)) / (np.max(emg) - np.min(emg))
        emg = noise(emg)
        if(len(emg)>l):
          z=z+1
          cut = int((len(emg)-l)/2)
          for i in range(cut):
            emg=np.delete(emg, 0, 0)
          for i in range (len(emg)-l):
            emg=np.delete(emg, l, 0)
        for i in range(0, len(emg)-WINDOW_SIZE, WINDOW_SLIDE):
            X_train.append(emg[i:i+WINDOW_SIZE, :])
            y_train.append(x["stimulus"])
print(z)
X_train = np.array(X_train)
y_train = np.array(y_train)

X_train.shape, y_train.shape

In [ ]:
X_test, y_test = [], []

for x in DATA_TEST:
    emg = x["emg"]
    emg = lpf(emg)
    emg = mlaw(emg)
    emg = (emg - np.min(emg)) / (np.max(emg) - np.min(emg))
    emg = noise(emg)
    if(len(emg)>l):
          z=z+1
          cut = int((len(emg)-l)/2)
          for i in range(cut):
            emg=np.delete(emg, 0, 0)
          for i in range (len(emg)-l):
            emg=np.delete(emg, l, 0)
    for i in range(0, len(emg)-WINDOW_SIZE, WINDOW_SLIDE):
        X_test.append(emg[i:i+WINDOW_SIZE, :])
        y_test.append(x["stimulus"])
X_test = np.array(X_test)
y_test = np.array(y_test)

X_test.shape, y_test.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(np.unique(y_train))
le.classes_

In [ ]:
!source /content/drive/MyDrive/Diplo2/semgs/bin/activate
from keras.utils import to_categorical

# Labels 1-52 should be mapped to 0-51 and then encoded
y_train = to_categorical(le.transform(y_train), 52)
y_test = to_categorical(le.transform(y_test), 52)


In [ ]:
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

In [ ]:
X_train.shape, y_train.shape


In [ ]:
X_test.shape, y_test.shape

In [ ]:
import random


indices = np.random.choice(len(X_train), len(X_train), False)
X_train = X_train[indices]
y_train = y_train[indices]

indices = np.random.choice(len(X_test), len(X_test), False)
X_test = X_test[indices]
y_test = y_test[indices]



In [ ]:
X_maj, y_maj = [], []

z=0
for x in DATA_TEST:
        emg = x["emg"]
        emg = lpf(emg)
        emg = mlaw(emg)
        emg = (emg - np.min(emg)) / (np.max(emg) - np.min(emg))
        emg = noise(emg)
        if(len(emg)>l):
          cut = int((len(emg)-l)/2)
          for i in range(cut):
            emg=np.delete(emg, 0, 0)
          for i in range (len(emg)-l):
            emg=np.delete(emg, l, 0)

        X_maj.append(emg)
        y_maj.append(x)

X_maj = np.array(X_maj)
y_maj = np.array(y_maj)
X_maj.shape, y_maj.shape

In [ ]:
def windowed_array(arr, size=WINDOW_SIZE, slide=WINDOW_SLIDE):
    W= len(arr)
    windows = []
    for i in range(0, W - size, slide):
        window = arr[i:i + size]
        windows.append(window[:, :, np.newaxis])
    return np.stack(windows)

In [ ]:
from keras.utils import Sequence
from scipy.stats import mode
from sklearn.metrics import accuracy_score
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from keras.metrics import *
from keras.utils import Sequence
import keras
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle

class CustomSequenceGenerator(Sequence):
    def __init__(self, data_list, labels, batch_size=4, num_classes=52, shuffle_data=True):
        self.data_list = data_list
        self.labels = labels
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle_data = shuffle_data
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.data_list) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x = [windowed_array(self.data_list[i]) for i in batch_indices]
        batch_y_dicts = [self.labels[i] for i in batch_indices] # Get the list of dictionaries

        # Stack all windows together for training
        batch_x = np.concatenate(batch_x, axis=0)

        # Extract the 'stimulus' value from each dictionary and repeat for each window
        batch_y_labels = [y_dict['stimulus'] for y_dict in batch_y_dicts]
        # Adjust labels to be zero-indexed
        batch_y_repeated = np.repeat([label - 1 for label in batch_y_labels], batch_x.shape[0] // len(batch_y_labels)) # Repeat label for each window


        return batch_x, tf.keras.utils.to_categorical(batch_y_repeated, num_classes=self.num_classes) # Use the extracted labels

    def on_epoch_end(self):
        self.indices = np.arange(len(self.data_list))
        if self.shuffle_data:
            self.indices = shuffle(self.indices)

In [ ]:
def evaluate_with_majority_voting(model, X_test, y_test, loss_fn=keras.losses.CategoricalCrossentropy()):
    """
    Evaluate model performance using majority voting on a test set, and also compute loss."""

    y_pred = []
    true_labels_list = []
    total_loss = 0
    y_prob_avv = []
    num_samples = len(X_test)
    num_classes = model.output_shape[-1]

    # Loop through each test sample
    for array, y_dict in zip(X_test, y_test):
        # Window the array
        windows = windowed_array(array)  # Shape: [num_windows, 15, 10, 1]

        # Get model predictions
        probs = model.predict(windows, verbose=0)  # Shape: [num_windows, 52]

        # Compute per-window predictions (argmax over classes)
        preds = np.argmax(probs, axis=1)

        # Majority voting
        voted = mode(preds, keepdims=False).mode
        y_pred.append(voted)

        # Extract and adjust the true label
        true_label = y_dict['stimulus'] - 1 # Adjust label to be zero-indexed
        true_labels_list.append(true_label)

        avg_prob = np.mean(probs, axis=0)
        y_prob_avv.append(avg_prob)
        # Calculate loss for the current sample using the true label for its window
        true_label_repeated = np.repeat(true_label, windows.shape[0])
        true_label_one_hot = tf.keras.utils.to_categorical(true_label_repeated, num_classes=52)
        batch_loss = loss_fn(true_label_one_hot, probs).numpy()  # loss per window
        total_loss += np.mean(batch_loss) # Take the mean loss across windows for the sample

    # Compute average loss for the entire test set
    avg_loss = total_loss / num_samples
    true_labels_one_hot = tf.keras.utils.to_categorical(true_labels_list, num_classes=num_classes)
    # Compute majority voting accuracy using the adjusted true labels
    acc = accuracy_score(true_labels_list, y_pred)
    top3acc_tensor = tf.keras.metrics.top_k_categorical_accuracy(true_labels_one_hot, np.array(y_prob_avv), k=3)
    top3acc = np.mean(top3acc_tensor.numpy())
    top5acc_tensor = tf.keras.metrics.top_k_categorical_accuracy(true_labels_one_hot, np.array(y_prob_avv), k=5)
    top5acc = np.mean(top5acc_tensor.numpy())
    print(f"Majority Voting Test Accuracy: {acc:.4f}")
    print(f"Majority Voting Test Top-3 Accuracy: {top3acc:.4f}")
    print(f"Majority Voting Test Top-5 Accuracy: {top5acc:.4f}")
    print(f"Average Test Loss: {avg_loss:.4f}")
    return acc, avg_loss, top3acc, top5acc

In [ ]:
def predict_with_majority_voting(model, X_data,y_true):
    """
    Predict class for each full array using majority vote over its windows.

    """
    y_pred = []

    for array in X_data:
        windows = windowed_array(array)  # shape: (num_windows, 15, 10, 1)
        probs = model.predict(windows, verbose=0)
        preds = np.argmax(probs, axis=1)
        voted = mode(preds, keepdims=False).mode
        y_pred.append(voted)

    acc = accuracy_score(y_true, y_pred)
    print(f"Predict Majority Voting Accuracy: {acc:.4f}")

    return np.array(y_pred)

#Time 45

In [ ]:
import gc
import keras
from keras import optimizers
from keras import callbacks
from keras.backend import clear_session
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import confusion_matrix

file = "/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time_45/"
if not os.path.exists(file):
    os.makedirs(file)
file = "/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time_45/"
if not os.path.exists(file):
    os.makedirs(file)


keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="auto", min_delta=0.001)
checkpoint_filepath="/content/drive/MyDrive/Diplo2/first_model.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max',
                                            save_best_only=True)
optimizer = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)



batch_size = [64,128,256,512]
epochs = [10,20,30,50]
batch_size = batch_size[0]
epochs = epochs[3]
loss = 'categorical_crossentropy'


model = Atzorinet_Time2((45,10,1))


model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', top3acc, top5acc])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback,model_checkpoint_callback], shuffle = True)
print("Evaluate on test data for Atzorinet_Tim_45 with batch={} and epochs={}".format(batch_size, epochs))
model = keras.models.load_model("/content/drive/MyDrive/Diplo2/first_model.keras",custom_objects={'top3acc': top3acc, 'top5acc': top5acc})
results = model.evaluate(X_test, y_test, batch_size=1)
print("Atzorinet_Tim_45 with batch={} and epochs={}".format(batch_size, epochs))
print("test loss, test ac, top3, top5:", results)
y_prediction = model.predict(X_test)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test_original = y_test
y_test=np.argmax(y_test, axis=1)
#Create confusion matrix and normalizes it over predicted (columns)
cm = confusion_matrix(y_test, y_prediction)
cm=np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time_45/{}".format(s),cm)
y_test = y_test_original
evaluate_with_majority_voting(model, X_maj, y_maj)
y_true = np.array([y_dict['stimulus'] - 1 for y_dict in y_maj])
y_val_pred = predict_with_majority_voting(model, X_maj, y_true)
cm = confusion_matrix(y_true, y_val_pred)
cm = np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time_45/{}".format(s),cm)

del model

clear_session()
gc.collect()

#Time_45 Bidir

In [ ]:
import gc
import keras
from keras import optimizers
from keras import callbacks
from keras.backend import clear_session
import tensorflow as tf
from keras import backend as K
from sklearn.metrics import confusion_matrix

file = "/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time_Bidir45/"
if not os.path.exists(file):
    os.makedirs(file)
file = "/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time_Bidir45/"
if not os.path.exists(file):
    os.makedirs(file)


keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=0,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=False,
    start_from_epoch=0,
)

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, mode="auto", min_delta=0.001)
checkpoint_filepath="/content/drive/MyDrive/Diplo2/first_model.keras"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', mode='max',
                                            save_best_only=True)
optimizer = optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
    weight_decay=None,
    clipnorm=None,
    clipvalue=None,
    global_clipnorm=None,
    use_ema=False,
    ema_momentum=0.99,
    ema_overwrite_frequency=None,
    loss_scale_factor=None,
    gradient_accumulation_steps=None,
    name='adam',
)



batch_size = [64,128,256,512]
epochs = [10,20,30,50]
batch_size = batch_size[0]
epochs = epochs[3]
loss = 'categorical_crossentropy'


model = Atzorinet_Time_Bidir2((45,10,1))


model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy', top3acc, top5acc])
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), callbacks=[callback,model_checkpoint_callback], shuffle = True)
print("Evaluate on test data for Atzorinet_Tim_45 with batch={} and epochs={}".format(batch_size, epochs))
model = keras.models.load_model("/content/drive/MyDrive/Diplo2/first_model.keras",custom_objects={'top3acc': top3acc, 'top5acc': top5acc})
results = model.evaluate(X_test, y_test, batch_size=1)
print("Atzorinet_Tim_45 with batch={} and epochs={}".format(batch_size, epochs))
print("test loss, test ac, top3, top5:", results)
y_prediction = model.predict(X_test)
y_prediction = np.argmax (y_prediction, axis = 1)
y_test_original = y_test
y_test=np.argmax(y_test, axis = 1)
#Create confusion matrix and normalizes it over predicted (columns)
cm = confusion_matrix(y_test, y_prediction)
cm=np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion/Atzorinet_Time_Bidir45/{}".format(s),cm)
y_test = y_test_original
evaluate_with_majority_voting(model, X_maj, y_maj)
y_true = np.array([y_dict['stimulus'] - 1 for y_dict in y_maj])
y_val_pred = predict_with_majority_voting(model, X_maj, y_true)
cm = confusion_matrix(y_true, y_val_pred)
cm = np.array(cm)
np.save("/content/drive/MyDrive/Diplo2/Confusion_Majority/Atzorinet_Time_Bidir45/{}".format(s),cm)

del model

clear_session()
gc.collect()